In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from astropy import units as u
from astropy import constants as const
from scipy import stats
from matplotlib.widgets import Slider, Button, RadioButtons
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [4]:
lumin_lelli, lumin_lelli_err, sig_lelli, sig_lelli_errpl, sig_lelli_errmin, distance= np.loadtxt('luminosities and sigmas.csv',  delimiter=',', skiprows = 1, unpack = True)
lumin_lelli = 10**lumin_lelli #10**log(lumin)
lumin_lelli_err = 10**lumin_lelli_err #10**log(lumin_err)

mass_bary = np.loadtxt('ellip_darkmatter.csv', usecols = (4), delimiter=',', skiprows = 1, unpack = True)
mass_bary = 10**(mass_bary*1.2) #m_bary in M_sun

ml_theory = 524.72*mass_bary**(-.5)
lumin_theory = mass_bary/ml_theory
vc_theory = (mass_bary/45)**(1/4)
sig_theory = vc_theory/np.sqrt(3)


In [5]:
def my_function(user_rand, user_syst, user_systerr):
    #return user_rand, user_syst, user_systerr

    err_random = np.empty(len(sig_lelli))
    err_systematic = np.empty(len(sig_lelli))
    err_systematicerr = np.empty(len(sig_lelli))

    for x in range(len(err_random)):
        err_random[x] = np.random.normal(0,1,1) * sig_lelli[x]*user_rand
        err_systematic[x] = user_syst
        err_systematicerr[x] = np.random.normal(0,1,1) *user_systerr
    
    x = np.log10(sig_theory)
    y = np.log10(lumin_theory)
    coeff=np.polyfit(x,y,1)
    
    sig_theory_lelli = np.empty(len(sig_lelli))
    for x in range(len(sig_lelli)):
        sig_theory_lelli[x] = 10**(np.log10(lumin_lelli[x])/coeff[0] -  coeff[1]/coeff[0])
    user_sig = sig_theory_lelli + np.sqrt(err_random**2 + err_systematic**2 + err_systematicerr**2)   
    
    random_arr = np.sort(user_sig)
    lelli_arr = np.sort(sig_lelli)
    cdf_random = np.cumsum(random_arr) / np.cumsum(random_arr)[-1] #CDF of random values
    cdf_lelli = np.cumsum(lelli_arr) / np.cumsum(lelli_arr)[-1] #CDF of Stacy's values

    ks_tot = stats.ks_2samp(random_arr, lelli_arr)
    print('KS for all values. stat: {:.4f}, p-value: {:.4f}'.format(ks_tot[0], ks_tot[1]))

    ttest = stats.ttest_ind(user_sig, sig_lelli)
    print('T-test for all values. stat: {:.4f}, p-value: {:.4f}'.format(ttest[0], ttest[1]))
    
    
    plt.figure(figsize=(12,10))
    plt.plot(sig_theory, lumin_theory, ls = '-.', alpha = 0.5)
    
    plt.scatter(sig_lelli, lumin_lelli, color = 'red')
    plt.errorbar(sig_lelli, lumin_lelli, yerr  = lumin_lelli_err, xerr = [sig_lelli_errmin, sig_lelli_errpl], fmt='o', alpha = 0.5, label = "Lelli's values", color = 'red')
    plt.scatter(user_sig, lumin_lelli, label = '"messed up" values')
    #plt.plot(10**(np.unique(x)), 10**(np.poly1d(np.polyfit(x, y, 1))(np.unique(x))))
    plt.xlabel('$\sigma_{los}$ (km/s)', fontsize = 'large')
    plt.ylabel('Lsun (M_sun)', fontsize = 'large')
    plt.legend(fontsize = 'large')
    plt.loglog()
    print('value for user_rand is the fraction of sigma value to multiply Gaussian by so user_rand = 1 means error in sigma is 100%.')
    print('value for user_syst is the systematic error in km/s.')
    print('value for user_systerr is the error in systematic error (i.e. +/- 2 kpc) multiplied by Gaussian.')
    plt.show()
uservals = interact(my_function, user_rand=widgets.FloatSlider(min=-10, max=10.0, step=0.1, continuous_update=False), user_syst=widgets.FloatSlider(min=-10, max=10.0, step=0.1, continuous_update=False), user_systerr=widgets.FloatSlider(min=-10, max=10.0, step=0.1, continuous_update=False))

interactive(children=(FloatSlider(value=0.0, continuous_update=False, description='user_rand', max=10.0, min=-…